In [9]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import re

In [2]:
answers = pd.read_csv('./MTC508/discussion_answers.csv')

In [3]:
questions = pd.read_csv('./MTC508/discussion_questions.csv')

In [4]:
answers.head()

,discussion_answer_id,umich_discussions_user_id,course_id,discussion_answer_content,discussion_question_id,discussion_answer_parent_discussion_answer_id,discussion_answer_created_ts,discussion_answer_updated_ts,on_demand_session_id,course_branch_id
0,ZroNsxc4Ee2C5xKrTYcc7w,1a77c68d3e75f92e3eb7addd5b0e99875a3cee27,H2xU8YQeEeqO7w5jjw7X2w,<text>The event I picked was the treaty betwee...,Loj5KgVZEe21VQ4xyvwaNw,NaN,2022-08-08 16:37:32.744,2022-08-08 16:37:32.744,NaN,branch~hkF_sO5LEeyxKgrNizMxaQ
1,2pQvXRv3Ee2FOwrnFlqg5Q,db6d799619ffeccbf051e29edaa465bc3bef5b18,H2xU8YQeEeqO7w5jjw7X2w,"<text>The government created this ""End of Trea...",Loj5KgVZEe21VQ4xyvwaNw,NaN,2022-08-14 17:38:05.638,2022-08-14 17:38:05.638,NaN,branch~hkF_sO5LEeyxKgrNizMxaQ
2,2SAgMAbiEe2KXA5zGGl7GQ,5a337f5e59a38095ed18d4cc487f3d1f75a78ef6,H2xU8YQeEeqO7w5jjw7X2w,<text>I actually took a look at the 1968 congr...,Loj5KgVZEe21VQ4xyvwaNw,NaN,2022-07-18 21:44:49.35,2022-07-25 02:11:00.708,NaN,branch~hkF_sO5LEeyxKgrNizMxaQ
3,U_HKABXXEe2toRLQeik5bQ,9233f9805b20b5b77a61987c226185a772dbf397,H2xU8YQeEeqO7w5jjw7X2w,<text>The postal act of 1792 established the r...,Loj5KgVZEe21VQ4xyvwaNw,NaN,2022-08-06 22:30:08.888,2022-08-06 22:30:08.888,NaN,branch~hkF_sO5LEeyxKgrNizMxaQ
4,r1N-_RXXEe2KXA5zGGl7GQ,9233f9805b20b5b77a61987c226185a772dbf397,H2xU8YQeEeqO7w5jjw7X2w,<text>I totally agree. The IRA should absolutl...,Loj5KgVZEe21VQ4xyvwaNw,l3V4WwxIEe2FkAoJNY_lJQ,2022-08-06 22:32:42.20,2022-08-06 22:32:42.20,NaN,branch~hkF_sO5LEeyxKgrNizMxaQ


In [5]:
questions.head()

,discussion_question_id,umich_discussions_user_id,discussion_question_title,discussion_question_details,discussion_question_context_type,course_id,course_module_id,course_item_id,discussion_forum_id,country_cd,group_id,discussion_question_created_ts,discussion_question_updated_ts
0,A6HMx5dwEe2d8BI-HmxRYQ,2246b3c08e2f3ee5d17f125ccc70baf3691667f9,Week 3,"First Item: ""[Optional] US History Videos""<tex...",weekContext,H2xU8YQeEeqO7w5jjw7X2w,NaN,NaN,SQ9BXDkbEe2-Bg5EvlLRdQ,NaN,NaN,2023-01-18 20:38:07.143,2023-01-20 04:28:44.755
1,2zX9kyJ1Ee2kCRKWCSdoqQ,5bc415ea72ee4b89c52b0db82c4c20f3b5a209ca,question about the live session,<text>Hello all</text><text>C​ould some help m...,weekContext,H2xU8YQeEeqO7w5jjw7X2w,NaN,NaN,Tkh5XxyVEeydTRLJergcdQ,NaN,NaN,2022-08-22 23:55:10.264,2022-08-22 23:55:10.264
2,AjMNQy0iEeu1IwoKYY9UtQ,04d598c7163af36ce85b21f3a136ef51ec7e4987,Completing modules early,"<text>Hello, </text><text>Is it possible to st...",weekContext,H2xU8YQeEeqO7w5jjw7X2w,NaN,NaN,4vPVtv07EeqGtA51XVU1kw,NaN,NaN,2020-11-23 00:22:45.399,2020-11-24 19:22:07.081
3,ExqKVOLkEeyhlwpGw76VXQ,6712d4528affb030d784cdab45ce2c92f890d75d,"Help, I am working this course and originally ...","<text>Hello, I am working with an old laptop a...",weekContext,H2xU8YQeEeqO7w5jjw7X2w,NaN,NaN,Tkh5XxyVEeydTRLJergcdQ,NaN,NaN,2022-06-03 02:22:54.259,2022-06-03 02:22:54.259
4,9S2e2JozEeyqrAoycocc8w,a5b7c8cb596ca7d3315cdb01a06f4e283e1cced9,I have reviewed all my assignments.,<text>Can you review my submission for me I ha...,weekContext,H2xU8YQeEeqO7w5jjw7X2w,NaN,NaN,Io3fiANCEeuOWQ4SUqFocw,NaN,NaN,2022-03-02 14:20:48.908,2022-03-04 20:12:31.298


In [6]:
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

In [10]:
def remove_html_tags(text):
    clean_text = re.sub(r'<[^>]*>', '', text)  # Remove HTML tags
    clean_text = re.sub(r'[^\w\s]', '', clean_text)  # Remove special characters
    return clean_text

In [14]:
sample = remove_html_tags(questions['discussion_question_details'][1])

In [15]:
embeddings = model.encode([sample])
print(embeddings)

[[-1.81708299e-02 -4.82319817e-02 -1.81827007e-03  1.82258245e-02
   1.14212157e-02  1.49547346e-02 -1.45491967e-02 -4.24731560e-02
  -4.78261150e-02  1.37570853e-04 -1.16764409e-02 -7.94394240e-02
   6.85168803e-02  1.18213501e-02 -8.26012809e-04 -4.56354804e-02
   9.36931744e-03 -4.20922507e-03 -1.05361886e-01  1.65609177e-02
   2.35735886e-02  5.82827954e-03 -5.73918223e-02 -4.69571445e-03
   2.01867335e-02  1.80068295e-02 -1.62802842e-02 -2.57757660e-02
   1.16716344e-02 -1.92629881e-02  5.68855926e-02 -1.28949834e-02
  -7.11533660e-03  3.90058681e-02  1.70660098e-06 -1.12727396e-02
   2.92018112e-02  5.52169010e-02 -5.84217608e-02  5.26437871e-02
  -5.99477328e-02  1.28809586e-02 -3.03060748e-03  2.79712863e-02
  -7.71471113e-02  7.42503582e-03 -1.26147978e-02 -1.13033624e-02
   4.20916900e-02  3.28311883e-02 -1.99007243e-02  2.96959169e-02
   6.98404238e-02  2.54098214e-02 -1.46227600e-02 -4.84239385e-02
  -1.18284775e-02 -2.31771972e-02  2.53951084e-02 -3.31680663e-02
   6.42960

In [17]:
len(embeddings[0])

768